In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 4.8 MB/s eta 0:00:00


In [1]:
# ===================================================================
#  Library
# ===================================================================
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import math
import time


from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_absolute_percentage_error
from tqdm.auto import tqdm

import warnings
warnings.simplefilter("ignore")

import unicodedata
import lightgbm as lgb

import optuna
import tensorflow as tf
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.layers import Dense, PReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping

In [2]:
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/train.csv')

In [3]:
kun_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/kun_exp00052.csv')
kun_oof = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/kun_exp00052_oof_pred.csv')
yuji_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/exp050.csv')
yuji_oof = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/oof_df_exp050.csv')

In [4]:
df = pd.concat([kun_oof.merge(yuji_oof, on='id'), df_train['price']], axis=1)
test = kun_test.merge(yuji_test, on='id')

In [17]:
df

,id,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8,...,pred_21,pred_22,pred_23,pred_24,pred_25,pred_26,pred_27,pred_28,pred_29,price
0,0,6881.892301,6517.414858,7476.567746,7052.610590,6809.002174,6465.089852,7714.185588,7010.855590,7184.306903,...,8613.396141,10717.468387,9447.344769,7984.506685,8272.276158,5876.657800,6016.322090,7374.013382,8496.804984,27587
1,1,3740.634027,3496.012582,3789.907986,3322.844722,3603.935425,3485.623398,3507.497452,3714.057283,3488.353022,...,4112.248835,3360.300264,3807.072052,3406.217022,3615.842705,4245.317355,4453.437876,4408.398403,3900.820201,4724
2,2,2954.247573,2735.110086,3288.647709,2893.840210,3253.350282,2885.690295,3202.103770,2799.365349,2989.931642,...,3167.730032,2929.953707,2957.126627,3618.763534,3022.052454,3913.586712,3705.560742,2921.102463,3116.785537,10931
3,3,8430.949224,8337.416095,9033.353098,8081.404050,8187.453186,8477.542430,8546.771462,8388.923127,8119.183310,...,7947.686753,8057.312440,8250.578545,8052.200549,8115.355025,8157.420044,7434.588221,8284.727211,7855.624335,16553
4,4,3972.418866,4254.790314,4089.549517,4272.046434,4446.949951,4022.828501,4374.413436,4369.679027,4150.709500,...,4200.673767,4535.020561,4420.531149,4421.192901,4254.960815,4031.139467,3972.143434,4027.789862,4720.410259,5158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27527,27527,12650.381284,13146.560575,12653.178198,11322.272339,13596.740391,12324.404433,11482.192363,11724.735381,11434.271326,...,12057.001640,13412.874272,12007.199573,12629.505062,11171.113895,10396.341985,14458.313885,14216.093476,12448.334050,32212
27528,27528,6494.944663,6633.645836,6184.736099,6171.310930,6500.089660,6209.227748,6202.691850,6608.664580,6505.449763,...,6344.288278,6026.474362,7055.987961,6138.980267,6987.163770,9398.511437,6896.820540,6121.882704,6568.329712,5400
27529,27529,12231.954239,15018.441231,14539.286659,14360.762819,11463.997914,13025.753065,12083.865500,12834.191021,13653.370272,...,16020.269380,14682.474553,14787.659188,13376.032925,13611.897758,14921.987650,15042.231812,14166.439728,15120.009430,22227
27530,27530,6832.933932,6911.537235,6877.397725,7139.135069,6847.030473,6937.347570,6261.849519,6822.260414,6649.630136,...,6997.856606,8323.489037,6872.441408,6424.874952,6747.818100,7189.621305,7638.963008,6865.697187,6743.532957,3054


In [18]:
test

,id,kun_pred_0,kun_pred_1,kun_pred_2,kun_pred_3,kun_pred_4,kun_pred_5,kun_pred_6,kun_pred_7,kun_pred_8,...,pred_20,pred_21,pred_22,pred_23,pred_24,pred_25,pred_26,pred_27,pred_28,pred_29
0,27532,9462.117521,9616.369529,9455.621674,9411.321320,9439.296013,9339.256741,9496.255924,9169.724825,9408.697900,...,8916.874560,9138.275094,9312.076577,9211.745829,9498.316353,8687.940060,9208.293004,8356.975825,9330.770219,9692.941277
1,27533,5431.933504,5579.297671,5502.604659,5421.178000,5426.123259,5696.500918,5520.123463,5516.627145,5408.202246,...,5508.598523,5524.770227,5759.403447,5638.017983,5628.653785,6202.828111,4785.541393,5604.598269,5398.402382,5744.756995
2,27534,5563.460291,5543.298528,5644.606929,5568.560278,5470.429184,5637.070499,5705.703291,5364.698823,5388.325417,...,5506.078552,5390.600874,5432.630177,5378.844623,5732.867996,5793.773542,5632.035892,5905.933910,5561.372542,5972.185238
3,27535,19615.535011,19333.748566,19484.368965,19209.276232,19294.428362,19318.374103,19171.163948,16937.493371,17264.432164,...,20360.798126,19146.682299,17090.358841,18263.872975,17604.417956,17961.093949,12794.552170,20108.010799,19183.244173,19561.667694
4,27536,4165.521012,4132.487296,4262.280629,4212.731834,4118.160390,4143.661658,4292.907859,4212.163549,4187.348948,...,4550.212166,3989.678345,5322.743324,4245.195615,3830.999498,3871.618438,6658.960126,4595.955768,4849.609894,4540.888135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27532,55064,13342.552114,13667.855920,13413.909141,13440.577192,13313.740622,12874.275284,13745.622717,14408.987795,14279.202379,...,9823.251404,13452.186317,12280.179966,10014.768978,13841.399729,12156.012675,11113.945897,13684.552604,13453.234406,12058.316829
27533,55065,8482.824014,8544.580408,8316.495493,8741.763658,8683.826037,8274.076872,8320.918352,9910.438599,10146.887679,...,11215.002932,9608.711346,9416.906835,8761.781589,9408.583043,9972.958526,10205.926610,8689.446574,10127.392831,8915.794126
27534,55066,6235.427898,6279.775623,6330.773011,6197.646469,6290.777000,6289.493699,6198.501766,6305.310528,6316.055393,...,5307.977447,5567.878736,5442.195710,6072.441981,5394.629396,5613.617670,6456.335598,5536.471253,5446.296301,5800.008342
27535,55067,5198.181210,5218.070387,5204.748284,5161.087157,5189.810533,5238.875991,5187.003192,4991.335690,5103.491682,...,4927.390509,5421.294054,5053.042998,5146.113453,4919.544215,5329.443448,4729.350399,5490.082143,4927.573842,4721.779342


In [19]:
# ===================================================================
#  CFG
# ===================================================================
class CFG:
    seed = 42
    num_pred =  len(df.columns)-2 #予測の数を指定
    hidden_size = 16

In [20]:
# ===================================================================
#  Utils
# ===================================================================
def get_score(y_true, y_pred):
    """get MAPE score"""
    score = mean_absolute_percentage_error(y_true, y_pred)
    return score * 100

In [21]:
def mape_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    diff = tf.abs((y_true - y_pred) / tf.clip_by_value(tf.abs(y_true), 1e-9, float("inf")))
    return 100. * tf.reduce_mean(diff)

In [22]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_seed(42)  # ここでシード値を設定

In [23]:
# 交差検証関数の定義
def get_custom_cv(df, n_splits):
    df = df.sort_values(by="price", ignore_index=True)
    df["fold"] = [i for i in range(n_splits)] * (df.shape[0] // n_splits) \
                + [i for i in range(df.shape[0] % n_splits)]
    df = df.sort_values(by="id", ignore_index=True)

    for fold in range(n_splits):
        train_idx = df[df["fold"] != fold].index
        valid_idx = df[df["fold"] == fold].index
        yield train_idx, valid_idx

# CVの設定
n_splits = 8
cv = list(get_custom_cv(df, n_splits))

In [24]:
exclude_columns = ['id', 'price']
remaining_columns = [col for col in df.columns if col not in exclude_columns]

In [25]:
# 1. データセットの作成
X = df[remaining_columns].values
y = df['price'].values

In [26]:
#一つ目のモデル

In [27]:
# テストデータの特徴量の取得
test_features = test[remaining_columns].values

# 各フォールドでのテストデータの予測を格納するリスト
test_preds = []

# OOF (Out Of Fold) predictions
oof_preds = np.zeros_like(y)

for fold, (train_idx, valid_idx) in enumerate(cv):
    print(f"Fold {fold + 1}")

    # データの取得
    X_train, y_train = X[train_idx], y[train_idx]
    X_valid, y_valid = X[valid_idx], y[valid_idx]

    # モデルの作成と学習
    model = Sequential([
        Dense(CFG.hidden_size*3, input_dim=CFG.num_pred),
        PReLU(),
        Dense(CFG.hidden_size*2),
        PReLU(),
        Dense(CFG.hidden_size),
        PReLU(),
        Dense(1)
    ])

    model.compile(optimizer=Adam(), loss=mape_loss)

    # EarlyStoppingコールバックの定義
    early_stopping = EarlyStopping(monitor='val_loss', patience=100, verbose=1, restore_best_weights=True)

    model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=50, batch_size=32, verbose=1, callbacks=[early_stopping])

    # OOF predictions
    oof_preds[valid_idx] = model.predict(X_valid).reshape(-1)

    print(f"FINISHI: fold{fold} Score: {mean_absolute_percentage_error(y_valid, oof_preds[valid_idx]):.4f}")

    # テストデータの予測
    test_pred = model.predict(test_features).reshape(-1)
    test_preds.append(test_pred)

# Calculate overall OOF score
oof_score = mean_absolute_percentage_error(y, oof_preds)
print("=" * 50)
print(f"FINISHI: Whole OOF Score: {oof_score:.4f}")


Fold 1
Epoch 1/50
753/753 [==============================] - 3s 3ms/step - loss: 44.4935 - val_loss: 43.3057
Epoch 2/50
753/753 [==============================] - 2s 3ms/step - loss: 43.9947 - val_loss: 43.2873
Epoch 3/50
753/753 [==============================] - 3s 4ms/step - loss: 43.9399 - val_loss: 43.6010
Epoch 4/50
753/753 [==============================] - 3s 3ms/step - loss: 43.9252 - val_loss: 43.7940
Epoch 5/50
753/753 [==============================] - 3s 4ms/step - loss: 43.9190 - val_loss: 43.3511
Epoch 6/50
753/753 [==============================] - 2s 2ms/step - loss: 43.9455 - val_loss: 43.7186
Epoch 7/50
753/753 [==============================] - 2s 2ms/step - loss: 43.9129 - val_loss: 43.2458
Epoch 8/50
753/753 [==============================] - 2s 2ms/step - loss: 43.9112 - val_loss: 43.2824
Epoch 9/50
753/753 [==============================] - 2s 2ms/step - loss: 43.8786 - val_loss: 43.2551
Epoch 10/50
753/753 [==============================] - 2s 2ms/step - loss: 

In [ ]:
# すべてのフォールドの予測の平均を取る
test["pred"] = np.mean(test_preds, axis=0)

test[["id", "pred"]].to_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp45_submission_stacking.csv', index=False, header=None)